# Optuna: A hyperparameter optimization framework

### This is modified public notebook from Kaggle, showing how to use Optuna.
### I have modified it to do cross-validation within Optuna.

* [1.Basic Concepts](#chapter1)
* [2. Let's build our optimization function using optuna](#chapter2)
* [3. XGBoost using Optuna](#chapter3)
* [5. Submission](#chapter5)

* <h4> In This Kernel I will use an amazing framework called <b>Optuna</b> to find the best hyparameters of our XGBoost and CatBoost </h4>

**So, Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative, define-by-run style user API.<br> The code written with Optuna enjoys high modularity, and the user of Optuna can dynamically construct the search spaces for the hyperparameters.** 
* To learn more about Optuna check this [link](https://optuna.org/)

MP: Good fast introduction to Optuna: https://towardsdatascience.com/why-is-everyone-at-kaggle-obsessed-with-optuna-for-hyperparameter-tuning-7608fdca337c

# 1. Basic Concepts <a class="anchor" id="chapter1"></a>
So, We use the terms study and trial as follows:
* <b>Study</b> : optimization based on an objective function
* <b>Trial</b> : a single execution of the objective function

In [1]:
#import optuna 
import optuna

from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time, warnings
#from optuna.visualization.matplotlib import plot_param_importances

warnings.filterwarnings('ignore')

train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

train.head()

print(train.shape)
columns = [col for col in train.columns.to_list() if col not in ['id','target']]

data=train[columns]
target=train['target']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.15,random_state=4)

(300000, 16)


# 2. Let's build our optimization function using optuna <a class="anchor" id="chapter2"></a>

### The following optimization function uses XGBoostRegressor model, so it takes the following arguments:
* the data
* the target
* trial (How many executions we will do)  
### and returns
* RMSE (Root Mean Squared Rrror)

## Notes:
* Note that I used some XGBoostRegressor hyperparameters from Xgboost official site. 
* So if you like to add more parameters or change them, check this [link](https://xgboost.readthedocs.io/en/latest/parameter.html) 
* Also I used early_stopping_rounds to avoid overfiting
* to speedup the training process we can use the GPU or you can comment the first param argument (the training process will takes a lot of time by only using the cpu 😩) 

# 3. XGBoost using Optuna <a class="anchor" id="chapter3"></a>

In [2]:
def evaluate_model_rkf(model, X_df, y_df, n_splits=4, random_state=3):
    X_values = X_df.values
    y_values = y_df.values
    rkf = KFold(n_splits=n_splits, random_state=random_state)
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_t, X_v = X_values[train_index, :], X_values[test_index, :]
        y_t = y_values[train_index]
        model.fit(
            X_t, y_t,
        )
        y_pred[test_index] += model.predict(X_v)
    y_pred
    return np.sqrt(mean_squared_error(y_train, y_pred))


# First, try raw XGBoost to make sure that evaluate_model_rkf works:

model = XGBRegressor(tree_method = 'gpu_hist', gpu_id=0, max_depth=8, eta=0.03, n_estimators=200)
evaluate_model_rkf(model, X_train, y_train, n_splits=4, random_state=2)

0.703764751179221

In [3]:
def objective(trial, random_state=1, n_splits=4, n_jobs=-1, early_stopping_rounds=50):
    params = {
        "tree_method": 'gpu_hist',
        "gpu_id": 0,
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": 400,
        "max_depth": trial.suggest_int("max_depth", 2, 15),
        "learning_rate": trial.suggest_uniform("learning_rate", 0.01, 0.09),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.2, 1),
        "subsample": trial.suggest_uniform("subsample", 0.3, 1),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_uniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
    }

    X = X_train
    y = y_train
    
    model = XGBRegressor(**params)
    rkf = KFold(n_splits=n_splits, random_state=random_state)
    X_values = X.values
    y_values = y.values
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_A, X_B = X_values[train_index, :], X_values[test_index, :]
        y_A, y_B = y_values[train_index], y_values[test_index]
        model.fit(X_A, y_A, eval_set=[(X_B, y_B)],
            eval_metric="rmse", early_stopping_rounds=early_stopping_rounds, verbose = False)
        y_pred[test_index] += model.predict(X_B)
    return (mean_squared_error(y_train, y_pred, squared=False))



time1 = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
print('Total time ', time.time()-time1)

# display params
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

[I 2022-06-04 20:50:49,390] A new study created in memory with name: no-name-45ec484e-d2d9-4b46-a25d-467e9ffb5c37
[I 2022-06-04 20:51:11,948] Trial 0 finished with value: 0.6988289305582124 and parameters: {'max_depth': 8, 'learning_rate': 0.050063088418425904, 'colsample_bytree': 0.923988893216138, 'subsample': 0.7318991246605181, 'alpha': 2.8301777069291516, 'lambda': 3.036489613239245e-05, 'min_child_weight': 282.04926545776914}. Best is trial 0 with value: 0.6988289305582124.
[I 2022-06-04 20:51:26,108] Trial 1 finished with value: 0.6998709618170263 and parameters: {'max_depth': 6, 'learning_rate': 0.07869120023858178, 'colsample_bytree': 0.5141833134360554, 'subsample': 0.9438518356701528, 'alpha': 0.032726657699866726, 'lambda': 0.0014445901126299574, 'min_child_weight': 815.5820917327338}. Best is trial 0 with value: 0.6988289305582124.
[I 2022-06-04 20:51:40,528] Trial 2 finished with value: 0.7022471471647614 and parameters: {'max_depth': 6, 'learning_rate': 0.036216141798738

Total time  529.2130689620972
           max_depth : 15
       learning_rate : 0.04192502207851453
    colsample_bytree : 0.557020876933197
           subsample : 0.6312356746524119
               alpha : 2.486571722519996
              lambda : 1.1823471520888105e-07
    min_child_weight : 517.5475000809336
best objective value : 0.6983776513576055


In [14]:
optuna_hyperpars = study.best_params
optuna_hyperpars['tree_method']='gpu_hist'
optuna_hyperpars['gpu_id']=0
optuna_hyperpars['n_estimators']=400
#optuna_hyperpars
optuna_xgb = XGBRegressor(**optuna_hyperpars)
optuna_xgb.fit(X_train, y_train)
optuna_xgb.predict(X_test)
print('Optuna model:', mean_squared_error(optuna_xgb.predict(X_test), y_test, squared = False))


Optuna model: 0.6963909629721695


In [ ]:
# compare timing with GridSearchCV on XGBoost:

time1 = time.time()
xgbb = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
param_grid = {'n_estimators':[400], 'eta':[0.02, 0.04, 0.06], 'max_depth':[4,7,10,12,15]}
xgbm = GridSearchCV(xgbb, param_grid, cv=4, scoring='neg_root_mean_squared_error')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, time.time()-time1)


In [ ]:
optuna_hyperpars = study.best_params
optuna_hyperpars['tree_method']='gpu_hist'
optuna_hyperpars['gpu_id']=0
optuna_hyperpars['n_estimators']=400
#optuna_hyperpars
optuna_xgb = XGBRegressor(**optuna_hyperpars)
optuna_xgb.fit(X_train, y_train)

print('Optuna model:', mean_squared_error(optuna_xgb.predict(X_test), y_test, squared = False))
print('GS model:', mean_squared_error(xgbm.predict(X_test), y_test, squared = False))

In [ ]:
study.trials_dataframe()

# Let's do some Quick Visualization for Hyperparameter Optimization Analysis
#### Optuna provides various visualization features in optuna.visualization to analyze optimization results visually

In [ ]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
'''plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
went and which parts of the space were explored more.'''
optuna.visualization.plot_slice(study)

In [ ]:
#plot_contour: plots parameter interactions on an interactive chart. You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['alpha',
                            #'max_depth',
                            'lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

### Optuna with cross-validation code was developed using this: https://aetperf.github.io/2021/02/16/Optuna-+-XGBoost-on-a-tabular-dataset.html

# Let's create an XGBoostRegressor model with the best hyperparameters

In [ ]:
#Best_trial = study.best_trial.params
#Best_trial["n_estimators"], Best_trial["tree_method"] = 10000, 'gpu_hist'
#Best_trial

In [ ]:
#preds = np.zeros(test.shape[0])
#kf = KFold(n_splits=5,random_state=48,shuffle=True)
#rmse=[]  # list contains rmse for each fold
#n=0
#for trn_idx, test_idx in kf.split(train[columns],train['target']):
#    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
#    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
#    model = xgb.XGBRegressor(**Best_trial)
#    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
#    preds+=model.predict(test[columns])/kf.n_splits
#    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
#    print(f"fold: {n+1} ==> rmse: {rmse[n]}")
#    n+=1

In [ ]:
#np.mean(rmse)

# 5. Submission <a class="anchor" id="chapter5"></a>

In [ ]:
#sub['target']=preds
#sub.to_csv('submission.csv', index=False)